# Exploratory Data Analysis


This Notebook examines the data at a high (summary) level.

---------
```
Zach Wolpe
zachcolinwolpe@gmail.com
29 May 2021
```
---------

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
import sys
sys.path.append('../process data/')
from process_data_raw_class import batch_processing

In [9]:
with open('../data objects/batch_processing_object.pkl', 'rb') as file2:
    bp = pickle.load(file2)